In [ ]:
import feedparser;
import pika;
import json;
import time;
import mysql.connector;
from datetime import datetime

In [ ]:
rss_url = 'https://rss.nytimes.com/services/xml/rss/nyt/AsiaPacific.xml'
feed = feedparser.parse(rss_url)

In [ ]:
output = {
    'id':feed.entries[0].id,
    'title':feed.entries[0].title,
    'url':feed.entries[0].link,
    'author':feed.entries[0].author,
    'date':feed.entries[0].published
}

In [ ]:
feed

In [ ]:
dt = datetime.strptime(output['date'], "%a, %d %b %Y %H:%M:%S %z")
print(dt)

In [ ]:
connection = pika.BlockingConnection(pika.ConnectionParameters('192.168.0.202'))
channel = connection.channel()

In [ ]:
json.dumps(output)

In [ ]:
channel.basic_publish(exchange='news.rss', routing_key='nytime', body=json.dumps(output))


In [ ]:
for entrie in feed.entries:
    output = {
        'id':entrie.id,
        'title':entrie.title,
        'url':entrie.link,
        'author':entrie.author,
        'date':entrie.published
    }
    channel.basic_publish(exchange='news.rss', routing_key='nytime', body=json.dumps(output))


In [ ]:
conn = mysql.connector.connect(
    host="192.168.0.202",
    port=4000,
    user="root",
    password="password",
    database="news"
)

cursor = conn.cursor()


In [ ]:
sql = "INSERT INTO test (name) VALUES (%s)"
val = ["Alice"]
try:
    cursor.execute(sql, val)
    conn.commit()  # Always commit after changes
except Exception as e:
    conn.rollback()
    print("database error:", e)
finally:
    cursor.close()
    conn.close()